# Diagnostic Genome Analysis

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [ ]:
sample = snakemake.wildcards[0]
print("Analysis for sample: " + str(sample))

In [ ]:
import pandas
import numpy

# import annovar data
annovar_output = snakemake.input[2]

columns_of_interest = ["Chr", "Start", "End", "Ref", "Alt", 
                       "Func.refGene", "Gene.refGene", "avsnp138", 
                       "SIFT_score", "Polyphen2_HDIV_score", "Polyphen2_HDIV_pred",
                       "CLNDN", "CLNDISDB", "CLNSIG", "CLNSIG"]

datatypes = {'SIFT_score': numpy.float64, 
             'Polyphen2_HDIV_score': numpy.float64}

annovar = pandas.read_csv(annovar_output, sep="\t", na_values = '.', 
                          usecols=columns_of_interest, dtype=datatypes) 

## Variant Data Exploration

### Significant variants (based on SIFT and Polyphen2 HDIV scores)

In [ ]:
annovar = annovar.sort_values(by=['SIFT_score', 'Polyphen2_HDIV_score'], ascending = False)
annovar = annovar.fillna(0)
annovar.head(25)

### Manhattan plot of variants

In [146]:
from plotly.subplots import make_subplots
import plotly.graph_objects as plotly_go
import re 

# get all unique chromosomes used for analysis.
chromosomes = annovar["Chr"].unique()
# sorts chromosomes by their number.
chromosomes = sorted(chromosomes, key=lambda chromosome:
                     [int(number) if number.isdigit() else number 
                      for number in re.split(r'(\d+)', chromosome)])

# make manhattan plot of variants.
fig = make_subplots(rows=1, cols=len(chromosomes), shared_yaxes=True, horizontal_spacing=0)
axes_labels = [dict(x=0.5, y=-0.17, showarrow=False,
                    text="Chromosome", font_size=15,
                    xref="paper", yref="paper"),
               dict(x=-0.07, y=0.5,
                    showarrow=False,
                    text="SIFT Score", font_size=15, textangle=-90,
                    xref="paper", yref="paper")]

fig.update_layout(title_text="Manhattan Plot of annotated sample " + sample + "'s variants", 
                  title_x=0.5, 
                  annotations=axes_labels)

# plot variants per chromosome.
for i, chromosome in enumerate(chromosomes):
    chr_variant_coord = annovar["Start"][annovar["Chr"] == chromosome]
    chr_variant_sift = annovar["SIFT_score"][annovar["Chr"] == chromosome]
    chr_variant_avsnp138 = annovar["avsnp138"][annovar["Chr"] == chromosome]
    
    fig.add_trace(plotly_go.Scattergl(
        name = chromosome,
        x = chr_variant_coord,
        y = chr_variant_sift,
        text = chr_variant_avsnp138,
        hoverinfo = "text",
        mode = "markers"
    ), row=1, col=i + 1)
    
    fig.update_xaxes(title_text=chromosome, row=1, col=i + 1, showticklabels=False)
    
fig.show()

## Pipeline Benchmarks

In [177]:
import os
import plotly.express as px

time_created = []
labels = []
benchmark_secs = []

for file in os.listdir("runs/" + sample + "/benchmarks/"):
    if file.endswith(".txt"):
        # removes extension of file for label
        labels.append(file[:-4])
        file = "runs/" + sample + "/benchmarks/" + file
        time_created.append(os.path.getmtime(file))
        
        with open(file) as file_content:
            header = file_content.readline()
            benchmark = file_content.readline().split("	")
            
            seconds = benchmark[0]
            mean_load = benchmark[-1]
            benchmark_secs.append(seconds)

def sort_list_by_list(to_be_sorted, to_sort_by):
    """
    Sort list by the respective values of another list. 
    """
    return [element for _,element in sorted(zip(to_sort_by, to_be_sorted))]

# sort lists by creation of benchmark
labels = sort_list_by_list(labels, time_created)
benchmark_secs = sort_list_by_list(benchmark_secs, time_created)
# set group colors for tools run in pipeline
color_groups = [re.sub("_[^_]*$", "", label) for label in labels]

fig = px.bar(x=labels, y=benchmark_secs, color= color_groups, category_orders={"x": labels})
fig.update_layout(title_text="Benchmarks of tools run during analysis of " + sample, 
                  title_x=0.5, 
                  xaxis_title="",
                  yaxis_title="CPU Time (seconds)",
                  showlegend=True)

fig.show()